# MNIST Training using PyTorch and Step Functions

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

In [1]:
!pip install sagemaker --upgrade --ignore-installed --no-cache --user

    100% |████████████████████████████████| 296kB 34.5MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 34.3MB/s ta 0:00:01
    100% |████████████████████████████████| 20.2MB 103.5MB/s ta 0:00:01
    100% |████████████████████████████████| 1.3MB 109.3MB/s ta 0:00:01
    100% |████████████████████████████████| 26.1MB 93.4MB/s ta 0:00:011
    100% |████████████████████████████████| 61kB 67.1MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 80.0MB/s ta 0:00:01
    100% |████████████████████████████████| 5.9MB 92.8MB/s ta 0:00:01
    100% |████████████████████████████████| 593kB 90.9MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 56.1MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 118.8MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 117.5MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 114.1MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 85.9MB/s ta 0:00:01
    100% |██████

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install torch==1.3.1 torchvision==0.4.2 --upgrade --ignore-installed --no-cache --user

    70% |██████████████████████▌         | 516.8MB 68.0MB/s eta 0:00:043  9% |███                             | 68.9MB 101.8MB/s eta 0:00:07    22% |███████                         | 161.9MB 99.3MB/s eta 0:00:06    24% |████████                        | 181.4MB 98.6MB/s eta 0:00:06    37% |████████████                    | 276.4MB 103.6MB/s eta 0:00:05    51% |████████████████▍               | 375.5MB 85.7MB/s eta 0:00:05    57% |██████████████████▍             | 421.3MB 76.9MB/s eta 0:00:05    57% |██████████████████▌             | 425.9MB 100.8MB/s eta 0:00:04    64% |████████████████████▉           | 477.0MB 80.9MB/s eta 0:00:04

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    78% |█████████████████████████▏      | 578.5MB 85.0MB/s eta 0:00:02

    79% |█████████████████████████▍      | 582.3MB 87.6MB/s eta 0:00:02

    79% |█████████████████████████▋      | 586.8MB 67.2MB/s eta 0:00:03

...

    98% |███████████████████████████████▍| 720.1MB 83.4MB/s eta 0:00:01

    98% |███████████████████████████████▋| 724.4MB 109.7MB/s eta 0:00:01

    99% |███████████████████████████████▊| 729.1MB 104.3MB/s eta 0:00:01

    99% |████████████████████████████████| 733.8MB 115.3MB/s eta 0:00:01

    100% |████████████████████████████████| 734.6MB 107.2MB/s ta 0:00:01
    100% |████████████████████████████████| 10.2MB 50.8MB/s ta 0:00:01
    100% |████████████████████████████████| 20.2MB 96.1MB/s ta 0:00:011                    | 286kB 71.6MB/s eta 0:00:01
    100% |████████████████████████████████| 2.1MB 106.0MB/s ta 0:00:01
docker-compose 1.24.1 has requirement PyYAML<4.3,>=3.10, but you'll have pyyaml 5.3 which is incompatible.
docker-compose 1.24.1 has requirement requests!=2.11.0,!=2.12.2,!=2.18.0,<2.21,>=2.6.1, but you'll have requests 2.22.0 which is incompatible.
awscli 1.16.283 has requirement botocore==1.13.19, but you'll have botocore 1.14.9 which is incompatible.
awscli 1.16.283 has requirement PyYAML<5.2,>=3.10; python_version != "2.6" and python_version != "3.3", but you'll have pyyaml 5.3 which is incompatible.
awscli 1.16.283 has requirement rsa<=3.5.0,>=3.1.2, but you'll have rsa 4.0 which is incompatible.
awscli 1.16.283 has requirement s3transfer<0.3.0,>=0.2.0

Forcing `pillow==6.2.1` due to https://discuss.pytorch.org/t/cannot-import-name-pillow-version-from-pil/66096

In [3]:
!pip uninstall -y pillow

Uninstalling Pillow-7.0.0:
  Successfully uninstalled Pillow-7.0.0
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install pillow==6.2.1 --upgrade --ignore-installed --no-cache --user

    100% |████████████████████████████████| 2.1MB 87.0MB/s ta 0:00:011
docker-compose 1.24.1 has requirement PyYAML<4.3,>=3.10, but you'll have pyyaml 5.3 which is incompatible.
docker-compose 1.24.1 has requirement requests!=2.11.0,!=2.12.2,!=2.18.0,<2.21,>=2.6.1, but you'll have requests 2.22.0 which is incompatible.
awscli 1.16.283 has requirement botocore==1.13.19, but you'll have botocore 1.14.9 which is incompatible.
awscli 1.16.283 has requirement PyYAML<5.2,>=3.10; python_version != "2.6" and python_version != "3.3", but you'll have pyyaml 5.3 which is incompatible.
awscli 1.16.283 has requirement rsa<=3.5.0,>=3.1.2, but you'll have rsa 4.0 which is incompatible.
awscli 1.16.283 has requirement s3transfer<0.3.0,>=0.2.0, but you'll have s3transfer 0.3.2 which is incompatible.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Restart the Kernel to Recognize New Dependencies Above

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)

In [ ]:
!pip3 list

## Setup

### Add a policy to your SageMaker role in IAM

**If you are running this notebook on an Amazon SageMaker notebook instance**, the IAM role assumed by your notebook instance needs permission to create and run workflows in AWS Step Functions. To provide this permission to the role, do the following.

1. Open the Amazon [SageMaker console](https://console.aws.amazon.com/sagemaker/). 
2. Select **Notebook instances** and choose the name of your notebook instance
3. Under **Permissions and encryption** select the role ARN to view the role on the IAM console
4. Choose **Attach policies** and search for `AWSStepFunctionsFullAccess`.
5. Select the check box next to `AWSStepFunctionsFullAccess` and choose **Attach policy**

If you are running this notebook in a local environment, the SDK will use your configured AWS CLI configuration. For more information, see [Configuring the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html).

Next, create an execution role in IAM for Step Functions. 

### Create an execution role for Step Functions

You need an execution role so that you can create and execute workflows in Step Functions.

1. Go to the [IAM console](https://console.aws.amazon.com/iam/)
2. Select **Roles** and then **Create role**.
3. Under **Choose the service that will use this role** select **Step Functions**
4. Choose **Next** until you can enter a **Role name**
5. Enter a name such as `StepFunctionsWorkflowExecutionRole` and then select **Create role**


Attach a policy to the role you created. The following steps attach a policy that provides full access to Step Functions, however as a good practice you should only provide access to the resources you need.  

1. Under the **Permissions** tab, click **Add inline policy**
2. Enter the following in the **JSON** tab

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:CreateTransformJob",
                "sagemaker:DescribeTransformJob",
                "sagemaker:StopTransformJob",
                "sagemaker:CreateTrainingJob",
                "sagemaker:DescribeTrainingJob",
                "sagemaker:StopTrainingJob",
                "sagemaker:CreateHyperParameterTuningJob",
                "sagemaker:DescribeHyperParameterTuningJob",
                "sagemaker:StopHyperParameterTuningJob",
                "sagemaker:CreateModel",
                "sagemaker:CreateEndpointConfig",
                "sagemaker:CreateEndpoint",
                "sagemaker:DeleteEndpointConfig",
                "sagemaker:DeleteEndpoint",
                "sagemaker:UpdateEndpoint",
                "sagemaker:ListTags",
                "lambda:InvokeFunction",
                "sqs:SendMessage",
                "sns:Publish",
                "ecs:RunTask",
                "ecs:StopTask",
                "ecs:DescribeTasks",
                "dynamodb:GetItem",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem",
                "dynamodb:DeleteItem",
                "batch:SubmitJob",
                "batch:DescribeJobs",
                "batch:TerminateJob",
                "glue:StartJobRun",
                "glue:GetJobRun",
                "glue:GetJobRuns",
                "glue:BatchStopJobRun"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": "sagemaker.amazonaws.com"
                }
            }
        },
        {
            "Effect": "Allow",
            "Action": [
                "events:PutTargets",
                "events:PutRule",
                "events:DescribeRule"
            ],
            "Resource": [
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTrainingJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTransformJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTuningJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForECSTaskRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForBatchJobsRule"
            ]
        }
    ]
}
```

3. Choose **Review policy** and give the policy a name such as `StepFunctionsWorkflowExecutionPolicy`
4. Choose **Create policy**. You will be redirected to the details page for the role.
5. Copy the **Role ARN** at the top of the **Summary**

### Import the required modules 

Now import the required modules from the Step Functions SDK and AWS SageMaker, configure an S3 bucket, and get the AWS SageMaker execution role.

In [ ]:
import sagemaker
import stepfunctions
import logging

from stepfunctions.template.pipeline import TrainingPipeline

sagemaker_session = sagemaker.Session()
stepfunctions.set_stream_logger(level=logging.INFO)

In [ ]:
# SageMaker Execution Role
# You can use sagemaker.get_execution_role() if running inside sagemaker's notebook instance
sagemaker_execution_role = sagemaker.get_execution_role()
print(sagemaker_execution_role)

In [ ]:
# Note this will cause the following: `An error occurred (AccessDenied) when calling the AttachRolePolicy operation: User: arn:aws:sts::835319576252:assumed-role/AmazonSageMaker-ExecutionRole-20191006T135881/SageMaker is not authorized to perform: iam:AttachRolePolicy on resource: role AWSStepFunctionsFullAccess`
#!aws iam attach-role-policy --role-name AWSStepFunctionsFullAccess --policy-arn {sagemaker_execution_role}

## Data
### Copy the Training Data to Your Notebook Disk

In [ ]:
local_data_path = './data'

In [ ]:
from torchvision import datasets, transforms

normalization_mean = 0.1307
normalization_std = 0.3081

# download the dataset
# this will not only download data to ./mnist folder, but also load and transform (normalize) them
datasets.MNIST(local_data_path, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((normalization_mean,), (normalization_std,))
]))

In [ ]:
!ls -R {local_data_path}

### Upload the Data to S3 for Distributed Training Across Many Workers
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.

This is S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.

In [ ]:
bucket = sagemaker_session.default_bucket()
data_prefix = 'sagemaker/pytorch-mnist/data'

### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [ ]:
training_data_uri = sagemaker_session.upload_data(path=local_data_path, bucket=bucket, key_prefix=data_prefix)
print('Input spec (S3 path): {}'.format(training_data_uri))

## Train
### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [ ]:
!pygmentize src/mnist_pytorch.py

### Use Step Functions to run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.


In [ ]:
from sagemaker.pytorch import PyTorch
import time

#model_output_path = 's3://{}/sagemaker/pytorch-mnist/training-runs'.format(bucket)

mnist_estimator = PyTorch(
                  entry_point='mnist_pytorch.py',
                  source_dir='./src',
#                  output_path=model_output_path,
                  role=sagemaker_execution_role,
                  framework_version='1.3.1',
                  train_instance_count=1,
                  train_instance_type='ml.c5.2xlarge',
                  enable_sagemaker_metrics=True,
                  hyperparameters={
                    'epochs': 5,
                    'backend': 'gloo'
                  },
                  # Assuming the logline from the PyTorch training job is as follows:
                  #    Test set: Average loss: 0.3230, Accuracy: 9103/10000 (91%)
                  metric_definitions=[
                     {'Name':'test:loss', 'Regex':'Test set: Average loss: (.*?),'},
                     {'Name':'test:accuracy', 'Regex':'(.*?)%;'}
                  ]
)

### Build a training pipeline with the Step Functions SDK

A typical task for a data scientist is to train a model and deploy that model to an endpoint. Without the Step Functions SDK, this is a four step process on SageMaker that includes the following.

1. Training the model
2. Creating the model on SageMaker
3. Creating an endpoint configuration
4. Deploying the trained model to the configured endpoint

The Step Functions SDK provides the [TrainingPipeline](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/pipelines.html#stepfunctions.template.pipeline.train.TrainingPipeline) API to simplify this procedure. The following configures `pipeline` with the necessary parameters to define a training pipeline.


In [ ]:
# paste the StepFunctionsWorkflowExecutionRole ARN from above
workflow_execution_role = "arn:aws:iam::835319576252:role/StepFunctionsWorkflowExecutionRole"

In [ ]:
model_output_s3_bucket = '{}/sagemaker/tensorflow-mnist/training-runs'.format(bucket)
print(model_output_s3_bucket)

In [ ]:
pipeline = TrainingPipeline(
    estimator=mnist_estimator,
    role=workflow_execution_role,
    inputs=training_data_uri,
    s3_bucket=model_output_s3_bucket
)

### Visualize the pipeline

You can now view the workflow definition, and also visualize it as a graph. This workflow and graph represent your training pipeline.

#### View the workflow definition

In [ ]:
print(pipeline.workflow.definition.to_json(pretty=True))

#### Visualize the workflow graph

In [ ]:
pipeline.render_graph()

### Create and execute the pipeline on AWS Step Functions

Create the pipeline in AWS Step Functions with [create](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.create).

In [ ]:
pipeline.create()

Run the workflow with [execute](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.execute). A link will be provided after the following cell is executed. Following this link, you can monitor your pipeline execution on Step Functions' console.

In [ ]:
execution = pipeline.execute()

In [ ]:
execution.render_progress()